In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
from statsmodels.tsa.arima.model import ARIMA

In [7]:
# Load the CSV file
file_path = '/workspaces/codespaces-jupyter/Medical Inventory Optimization Dataset - Cleaned.csv'
data = pd.read_csv(file_path)

In [8]:
# Convert Dateofbill to datetime
data['Dateofbill'] = pd.to_datetime(data['Dateofbill'])

# Aggregate Quantity and Final_Sales by DrugName, Dateofbill, Specialisation, and Dept
aggregated_data = data.groupby(['DrugName', 'Dateofbill', 'Specialisation', 'Dept']).agg({
    'Quantity': 'sum',
    'Final_Sales': 'sum'
}).reset_index()

In [9]:
# Feature Engineering
# Extract date features
aggregated_data['Month'] = aggregated_data['Dateofbill'].dt.month
aggregated_data['Day'] = aggregated_data['Dateofbill'].dt.day
aggregated_data['Year'] = aggregated_data['Dateofbill'].dt.year
aggregated_data['Is_Weekend'] = aggregated_data['Dateofbill'].dt.dayofweek >= 5

# Cyclic features for Month
aggregated_data['Month_sin'] = np.sin(2 * np.pi * aggregated_data['Month'] / 12)
aggregated_data['Month_cos'] = np.cos(2 * np.pi * aggregated_data['Month'] / 12)

# Lag features for Quantity and Final_Sales
aggregated_data['Lag_7'] = aggregated_data.groupby('DrugName')['Final_Sales'].shift(7)


In [10]:

# Calculate rolling averages for Quantity and Final_Sales
aggregated_data['Quantity_MA'] = aggregated_data.groupby('DrugName')['Quantity'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
aggregated_data['Final_Sales_MA'] = aggregated_data.groupby('DrugName')['Final_Sales'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

# Drop NaN values after creating lag features
aggregated_data.dropna(inplace=True)

# Prepare data for modeling
X = aggregated_data[['Month_sin', 'Month_cos', 'Day', 'Year', 'Is_Weekend', 'Lag_7', 'Quantity_MA', 'Final_Sales_MA']]
y = aggregated_data['Final_Sales']

# Split data into training and testing sets using TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [11]:
# 1. Linear Regression with Ridge Regularization
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
ridge_predictions = ridge_model.predict(X_test)
ridge_rmse = mean_squared_error(y_test, ridge_predictions, squared=False)


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [12]:
# 2. Random Forest with Hyperparameter Tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'max_features': ['sqrt', 'log2']
}
grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=tscv, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_rf_model = grid_search.best_estimator_
rf_predictions = best_rf_model.predict(X_test)
rf_rmse = mean_squared_error(y_test, rf_predictions, squared=False)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [13]:
# 3. LSTM Model
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled, X_test_scaled = X_scaled[train_index], X_scaled[test_index]

X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

lstm_model = Sequential()
lstm_model.add(Input(shape=(1, X_train_scaled.shape[1])))
lstm_model.add(LSTM(50, return_sequences=True))
lstm_model.add(LSTM(50))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32)
lstm_predictions = lstm_model.predict(X_test_lstm)
lstm_rmse = mean_squared_error(y_test, lstm_predictions, squared=False)


Epoch 1/10


246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 392425.3125
Epoch 2/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 355692.9062
Epoch 3/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 348775.2812
Epoch 4/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 341629.8438
Epoch 5/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 300236.6875
Epoch 6/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 358179.6875
Epoch 7/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 383795.0938
Epoch 8/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 384680.1562
Epoch 9/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 347396.6562
Epoch 10/10
246/246 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 370441.4375
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [14]:
# 4. Stacking Regressor (Combining Ridge and Random Forest)
estimators = [
    ('rf', best_rf_model),
    ('ridge', ridge_model)
]

stacking_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stacking_model.fit(X_train, y_train)
stacking_predictions = stacking_model.predict(X_test)
stacking_rmse = mean_squared_error(y_test, stacking_predictions, squared=False)


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [15]:

# Print RMSE for each model
print(f'Ridge Regression RMSE: {ridge_rmse}')
print(f'Random Forest RMSE (with hyperparameter tuning): {rf_rmse}')
print(f'LSTM RMSE: {lstm_rmse}')
print(f'Stacking Regressor RMSE: {stacking_rmse}')

Ridge Regression RMSE: 230.68938445685066
Random Forest RMSE (with hyperparameter tuning): 231.38696831791785
LSTM RMSE: 373.09990166703517
Stacking Regressor RMSE: 230.6648075920067


In [16]:
# Calculate last known moving averages and lag for future predictions
last_known_date = aggregated_data['Dateofbill'].max()
last_known_quantity_ma = aggregated_data.loc[aggregated_data['Dateofbill'] == last_known_date, 'Quantity_MA'].values[0]
last_known_final_sales_ma = aggregated_data.loc[aggregated_data['Dateofbill'] == last_known_date, 'Final_Sales_MA'].values[0]
last_known_lag_7 = aggregated_data.loc[aggregated_data['Dateofbill'] == last_known_date, 'Lag_7'].values[0]



In [18]:
# Predict future sales using the best model (e.g., Stacking)
future_dates = pd.date_range(start=last_known_date, periods=30, freq='D')
future_data = pd.DataFrame({
    'Dateofbill': future_dates,
    'Month_sin': np.sin(2 * np.pi * future_dates.month / 12),
    'Month_cos': np.cos(2 * np.pi * future_dates.month / 12),
    'Day': future_dates.day,
    'Year': future_dates.year,
    'Is_Weekend': future_dates.dayofweek >= 5,
    'Lag_7': last_known_lag_7,
    'Quantity_MA': last_known_quantity_ma,
    'Final_Sales_MA': last_known_final_sales_ma
})

In [19]:
# Scale and reshape future data for LSTM
future_X = future_data[['Month_sin', 'Month_cos', 'Day', 'Year', 'Is_Weekend', 'Lag_7', 'Quantity_MA', 'Final_Sales_MA']]
future_X_scaled = scaler.transform(future_X)
future_X_lstm = future_X_scaled.reshape((future_X_scaled.shape[0], 1, future_X_scaled.shape[1]))

future_predictions = stacking_model.predict(future_X)  # or use lstm_model for future prediction
optimal_stock_levels = future_predictions * 1.1  # Adding 10% buffer

# Display future predictions and optimal stock levels
future_data['Predicted_Sales'] = future_predictions
future_data['Optimal_Stock_Level'] = optimal_stock_levels

print(future_data[['Dateofbill', 'Predicted_Sales', 'Optimal_Stock_Level']])

   Dateofbill  Predicted_Sales  Optimal_Stock_Level
0  2022-12-31       300.138249           330.152074
1  2023-01-01       294.915775           324.407353
2  2023-01-02       290.433817           319.477199
3  2023-01-03       290.611279           319.672406
4  2023-01-04       290.756168           319.831785
5  2023-01-05       290.944992           320.039491
6  2023-01-06       290.975642           320.073206
7  2023-01-07       295.550719           325.105791
8  2023-01-08       295.628144           325.190958
9  2023-01-09       291.271056           320.398162
10 2023-01-10       291.327352           320.460087
11 2023-01-11       291.476562           320.624218
12 2023-01-12       291.610457           320.771503
13 2023-01-13       291.705635           320.876198
14 2023-01-14       296.306339           325.936973
15 2023-01-15       296.435122           326.078634
16 2023-01-16       292.088823           321.297705
17 2023-01-17       292.140815           321.354897
18 2023-01-1